In [1]:
import numpy as np
from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import pandas as pd

In [2]:
client = EfdClient('summit_efd')
csc_index = 1

start = Time('2020-10-26T13:41:30') #this is UTC
end = start+timedelta(days=1)
#import astropy
#astropy.utils.data.clear_download_cache()

In [3]:
xmlFile = 'MTM2_Telemetry.xml'
fid = open(xmlFile)

topics = dict()
params = dict()
counts = dict()
lastT = dict()
values= dict()
nline = 0
for line in fid:
    if '<EFDB_Topic>' in line:
        topic = line[13:-14]
        t1, t2 = topic.split('_')
        dfa = await client.select_time_series('lsst.sal.%s.%s'%(t1,t2), '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
    elif '<EFDB_Name>' in line:
        param = line[21:-13]
        nline+=1
        idx = '%d'%nline
    elif '<Count>' in line:
        count = line[17:-9]
        #print(topic, param,count)
        topics[idx] = topic
        params[idx] = param
        counts[idx] = count
        
        c = int(count)
        lastT[idx] = pd.to_datetime(dfa.private_sndStamp[-1], unit='s')
        if c == 1:
            values[idx] = dfa[param][-1]
        else:
            values[idx] = np.mean([dfa['%s%d'%(param,i)][-1] for i in range(c)])
    #if nline>7:
    #    break
#print(nline)
fid.close()

In [6]:
frame = {'Topic': topics, 'Name': params,
         'Count': counts, 'Last Timestamp (UTC)': lastT,
         'value (average)': values}
d = pd.DataFrame(frame)

In [7]:
d

,Topic,Name,Count,Last Timestamp (UTC),value (average)
1,MTM2_mirrorPositionMeasured,xTilt,1,2020-10-26 20:47:56.097801924,1.0
2,MTM2_mirrorPositionMeasured,yTilt,1,2020-10-26 20:47:56.097801924,1.0
3,MTM2_mirrorPositionMeasured,piston,1,2020-10-26 20:47:56.097801924,1.0
4,MTM2_mirrorPositionMeasured,xPosition,1,2020-10-26 20:47:56.097801924,1.0
5,MTM2_mirrorPositionMeasured,yPosition,1,2020-10-26 20:47:56.097801924,1.0
6,MTM2_mirrorPositionMeasured,thetaZPosition,1,2020-10-26 20:47:56.097801924,1.0
7,MTM2_axialForcesMeasured,axialForceMeasured,72,2020-10-26 20:48:06.098780870,44.5
8,MTM2_tangentForcesMeasured,tangentLink0DegForceMeasured,1,2020-10-26 20:48:16.099709272,1.0
9,MTM2_tangentForcesMeasured,tangentLink60DegForceMeasured,1,2020-10-26 20:48:16.099709272,1.0
10,MTM2_tangentForcesMeasured,tangentLink120DegForceMeasured,1,2020-10-26 20:48:16.099709272,1.0
